# [Totient permutation](https://projecteuler.net/problem=70)

> Euler's Totient function, φ(*n*) \[sometimes called the phi function\], is used to determine the number of positive numbers less than or equal to *n* which are relatively prime to *n*. For example, as 1, 2, 4, 5, 7, and 8, are all less than nine and relatively prime to nine, φ(9)=6.
> The number 1 is considered to be relatively prime to every positive number, so φ(1)=1.
>
> Interestingly, φ(87109)=79180, and it can be seen that 87109 is a permutation of 79180.
>
> Find the value of *n*, 1 < *n* < 10<sup>7</sup>, for which φ(*n*) is a permutation of *n* and the ratio *n*/φ(*n*) produces a minimum.


For this problem, I’m demonstrating that Julia can be fast enough that a simple implementation of a solution can work without needing to reformulate it to be as fast as possible. For this one, I used the module-supplied totient function, but added a binding for `ϕ` just to make the code match the problem statement better. I needed an operator for the ‘is a permutation of’ relation. I chose `∽` from the `prec-comparison` group in [Julia’s parser code](https://github.com/JuliaLang/julia/blob/master/src/julia-parser.scm) because ‘permutation of’ reminds me of ‘similar to’ and I wanted something with the same precedence as other comparison operators.

In [1]:
using Primes
ϕ = totient
∽(a::Integer, b::Integer) = sort(digits(a)) == sort(digits(b))

∽ (generic function with 1 method)

Once those definitions are made, the problem can be expressed as a one-liner: create a dictionary of $n$ to $n/ϕ(n)$ for $n$s whose digits are a permutation of $ϕ(n)$’s. Pluck out the $n$ associated with the smallest ratio. Done.

In [2]:
@time argmin(Dict(n => n/ϕ(n) for n in 2:9999999 if n ∽ ϕ(n)))

 13.692762 seconds (115.84 M allocations: 7.746 GiB, 3.56% gc time)


8319823

Is this the fastest implementation out there? No. Is it easy to see that it is solving the problem as stated? Absolutely. You might think that computing `ϕ(n)` twice would slow things down, but I timed a version that only does it once, and it made no meaningful difference—the compiler took care of optimizing that, so I can keep the more natural expression. The path to faster execution is by transforming the problem, but that’s going to take me more than 13 seconds to do, and sometimes you just need an answer.

In [3]:
function p70a(m)
    argmin(Dict(n => n/ϕ(n) for n in 2:m-1 if n ∽ ϕ(n)))
end

@time p70a(10000000)

 13.003066 seconds (114.16 M allocations: 7.666 GiB, 3.31% gc time)


8319823

In [4]:
function p70b(m)
    A = Dict{Int64,Float64}()
    for n in 2:m-1
        d = ϕ(n)
        if n ∽ d
            A[n] = n/d
        end
    end
    argmin(A)
end

@time p70b(10000000)

 12.911443 seconds (114.13 M allocations: 7.664 GiB, 3.30% gc time)


8319823